# 천안정수장 원수 및 침전탁도

필요한 라이브러리 import

In [50]:
import streamlit as st
import pandas as pd
import seaborn as sns
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import openpyxl

plt.rcParams['font.family'] = 'Malgun Gothic'  # 또는 출력된 다른 이름
plt.rcParams['axes.unicode_minus'] = False  # 한글 폰트 사용 시, 마이너스 기호 깨짐 방지
%matplotlib inline


c:\Users\HB\miniconda3\envs\KPro_HB\lib\site-packages\IPython\core\events.py:82: UserWarning: Glyph 53441 (\N{HANGUL SYLLABLE TAG}) missing from font(s) DejaVu Sans.
  func(*args, **kwargs)
c:\Users\HB\miniconda3\envs\KPro_HB\lib\site-packages\IPython\core\events.py:82: UserWarning: Glyph 46020 (\N{HANGUL SYLLABLE DO}) missing from font(s) DejaVu Sans.
  func(*args, **kwargs)
c:\Users\HB\miniconda3\envs\KPro_HB\lib\site-packages\IPython\core\events.py:82: UserWarning: Glyph 50896 (\N{HANGUL SYLLABLE WEON}) missing from font(s) DejaVu Sans.
  func(*args, **kwargs)
c:\Users\HB\miniconda3\envs\KPro_HB\lib\site-packages\IPython\core\events.py:82: UserWarning: Glyph 47000 (\N{HANGUL SYLLABLE RAE}) missing from font(s) DejaVu Sans.
  func(*args, **kwargs)
c:\Users\HB\miniconda3\envs\KPro_HB\lib\site-packages\IPython\core\events.py:82: UserWarning: Glyph 48516 (\N{HANGUL SYLLABLE BUN}) missing from font(s) DejaVu Sans.
  func(*args, **kwargs)
c:\Users\HB\miniconda3\envs\KPro_HB\lib\site-packa

### 로그 변환할지 확인하기 위한 히스토그램 출력

In [ ]:
def check_to_log(df):
    # 로그 변환 대상 컬럼 리스트 (양수여야 함)
    columns_to_check = ['탁도', '약품주입율 계산']  # 필요시 다른 컬럼 추가

    # 히스토그램 출력
    for col in columns_to_check:
        fig, axs = plt.subplots(1, 2, figsize=(12, 4))
        
        # 원래 분포
        axs[0].hist(df[col].dropna(), bins=50, color='skyblue')
        axs[0].set_title(f'{col} - 원래 분포')
        
        # 로그 변환 분포
        log_data = df[col][df[col] > 0]  # 0 이하 제외
        log_data = np.log10(log_data)
        axs[1].hist(log_data.dropna(), bins=50, color='lightgreen')
        axs[1].set_title(f'{col} - 로그 변환 분포')

        plt.tight_layout()
        plt.show()

### 데이터 불러오기
### 및 데이터 전처리

In [ ]:
def load_data():
    # 데이터 불러오기
    df = pd.read_csv('./data/CA_Water_Quality.csv', encoding="CP949")

    # df["로그 탁도"] = np.log10(df["탁도"])
    # df["로그 응집제 주입률"] = np.log10(int(df["약품주입율 계산"]))
    # np.log10은 데이터 전처리 후 사용
    
    
    df.rename(columns={'약품주입율 계산': '약품주입율'}, inplace=True)


    X = df[
        [
            "탁도",
            "pH",
            "알칼리도",
            "전기전도도",
            "수온",
            "유입유량",
            "침전탁도",
            "약품주입율"
        ]
    ]

    # 두 컬럼만 숫자로 변환 (숫자가 아닌 값은 NaN으로 처리)
    df['탁도'] = pd.to_numeric(df['탁도'], errors='coerce')
    df['약품주입율 계산'] = pd.to_numeric(df['약품주입율 계산'], errors='coerce')

    # 변환 후 NaN 확인 (비정상값이 있었는지 확인 가능)
    print(df[['탁도', '약품주입율 계산']].isna().sum())

    # 탁도 0개, 약품주입율 계산 11개 데이터가 na로, 행 삭제

    df = df.dropna(subset=['탁도', '약품주입율 계산'])

    # 탁도 분포에 따라 로그 변환 실행
    df['로그 탁도'] = np.log10(df["탁도"])

    return X


In [54]:
df_raw = load_data()
df_raw.head()

C:\Users\HB\AppData\Local\Temp\ipykernel_8692\2249610441.py:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/CA_Water_Quality.csv', encoding="CP949")
c:\Users\HB\miniconda3\envs\KPro_HB\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


,탁도,pH,알칼리도,전기전도도,수온,유입유량,침전탁도,약품주입율
0,2.59,7.34,35.88,145.31,10.13,12557.0,0.33,16.0
1,2.63,7.36,35.88,145.16,10.13,12497.0,0.33,16.0
2,2.63,7.35,35.88,145.16,10.13,12498.0,0.32,16.0
3,2.63,7.31,35.88,145.31,10.13,12570.0,0.33,16.0
4,2.59,7.31,35.84,145.31,10.13,12614.0,0.34,16.0


## 예측 함수

In [56]:
def run(target, input, max_depth, n_estimators, learning_rate, subsample):

    Xt, Xts, yt, yts = train_test_split(input, target, test_size=0.2, shuffle=False)

    xgb = XGBRegressor(
        random_state=2, 
        n_jobs=-1,
        max_depth=max_depth,
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        subsample=subsample
    )

    xgb.fit(Xt, yt)
    
    yt_pred = xgb.predict(Xt)
    yts_pred = xgb.predict(Xts)

    mse_train = mean_squared_error(10**yt, 10**yt_pred)
    mse_test = mean_squared_error(10**yts, 10**yts_pred)
    st.write(f"학습 데이터 MSE: {mse_train}")
    st.write(f"테스트 데이터 MSE: {mse_test}")

    r2_train = r2_score(10**yt, 10**yt_pred)
    r2_test = r2_score(10**yts, 10**yts_pred)
    st.write(f"학습 데이터 R2: {r2_train}")
    st.write(f"테스트 데이터 R2: {r2_test}")

    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    ax = axes[0]
    print(Xt.columns)
    ax.scatter(Xt.iloc[:, 0], yt, s=3, label="학습 데이터 (실제)")
    ax.scatter(Xt.iloc[:, 0], yt_pred, s=3, label="학습 데이터 (예측)", c="r")
    ax.grid()
    ax.legend(fontsize=13)
    # ax.set_xlabel("로그 원수 탁도")
    # ax.set_ylabel("로그 응집제 주입률")
    
    ax.set_xlabel(input.columns[0])
    ax.set_ylabel(target.columns[0])

    ax.set_title(
        rf"학습 데이터  MSE: {round(mse_train, 4)}, $R^2$: {round(r2_train, 2)}",
        fontsize=18,
    )

    ax = axes[1]
    # ax.scatter(Xt["로그 원수 탁도"], yt, s=3, label="학습 데이터 (실제)")
    # ax.scatter(Xt["로그 원수 탁도"], yt_pred, s=3, label="학습 데이터 (예측)", c="r")
    
    ax.scatter(Xt.iloc[:, 0], yt, s=3, label="학습 데이터 (실제)")
    ax.scatter(Xt.iloc[:, 0], yt_pred, s=3, label="학습 데이터 (예측)", c="r")

    ax.grid()
    ax.legend(fontsize=13)

    ax.set_xlabel(input.columns[0])
    ax.set_ylabel(target.columns[0])

    ax.set_title(
        rf"테스트 데이터  MSE: {round(mse_test, 4)}, $R^2$: {round(r2_test, 2)}",
        fontsize=18,
    )

    st.pyplot(fig)

## 메인함수

In [ ]:
def main():

    # load data
    dff = load_data()

    # Select Variable
    st.markdown("## Select Target Variable")
    column = st.selectbox("Target 변수를 선택하세요.", dff.columns.tolist())
    col = dff[[column]]
    st.dataframe(dff[[column]].head())

    st.markdown("## Select Input Variables")
    input_columns = st.multiselect("복수의 컬럼을 선택하세요.", dff.columns.tolist())
    filtered_col = dff[input_columns]

    st.dataframe(dff[input_columns].head())

    # Hyperparameters
    max_depth = st.slider("Select max depth", min_value=0, max_value=20, value=3)
    n_estimators = st.slider("n_estimators", min_value=20, max_value=500, value=50)
    learning_rate = st.slider("learning_rate", min_value=0.00, max_value=1.00, step=0.01, value=0.1)
    subsample = st.slider("subsample", min_value=0.00, max_value=1.00, step=0.01, value=0.8)

    if st.button("차트 만들기"):
        run(col, filtered_col, max_depth, n_estimators, learning_rate, subsample)